In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from tqdm import tqdm, tqdm_notebook
from collections import defaultdict
from google.colab import drive

In [0]:
from google.colab import files


In [0]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
from google.cloud import bigquery
# TODO: Uncomment the line below to set the `project` variable.
project_id = 'mimiciii-database'
#
# The `project` variable defines the project to be billed for query
# processing. The user must have the bigquery.jobs.create permission on
# this project to run a query. See:
# https://cloud.google.com/bigquery/docs/access-control#permissions
client = bigquery.Client(project=project_id)
sql = """
    SELECT *
    FROM `mimiciii-database.MIMIC3_V1_4.ADMISSIONS`
    LIMIT 100
"""

# Run a Standard SQL query using the environment's default project
#df = client.query(sql).to_dataframe()""

# Run a Standard SQL query with the project set explicitly
project_id = 'mimiciii-database'
df = client.query(sql, project=project_id).to_dataframe()
#df

In [0]:
#All data points- first 24hrs
query6_4 = """
WITH sample_icu AS(
SELECT icustay_id
FROM `mimiciii-database.MIMIC3_V1_4.ICUSTAYS` 
WHERE dbsource = 'metavision'
--ORDER BY rand()
--LIMIT 50
)
  select ie.subject_id, ie.hadm_id, ie.icustay_id, ie.intime,ie.outtime,
  ce.charttime
  , case
    when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 1 -- HeartRate
    when itemid in (51,442,455,6701,220179,220050) and valuenum > 0 and valuenum < 400 then 2 -- SysBP
    when itemid in (8368,8440,8441,8555,220180,220051) and valuenum > 0 and valuenum < 300 then 3 -- DiasBP
    when itemid in (456,52,6702,443,220052,220181,225312) and valuenum > 0 and valuenum < 300 then 4 -- MeanBP
    when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 5 -- RespRate
    when itemid in (223761,678) and valuenum > 70 and valuenum < 120  then 6 -- TempF, converted to degC in valuenum call
    when itemid in (223762,676) and valuenum > 10 and valuenum < 50  then 6 -- TempC
    when itemid in (646,220277) and valuenum > 0 and valuenum <= 100 then 7 -- SpO2
    when itemid in (807,811,1529,3745,3744,225664,220621,226537) and valuenum > 0 and valuenum <1000 then 8 -- Glucose

    else null end as VitalID
      -- convert F to C
  , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

  from `mimiciii-database.MIMIC3_V1_4.ICUSTAYS` AS ie
  left join `mimiciii-database.MIMIC3_V1_4.CHARTEVENTS` ce
  on ie.icustay_id = ce.icustay_id
  and ce.charttime between ie.intime and ie.outtime
  and DATETIME_DIFF(ce.charttime, ie.intime, SECOND) > 0
  and DATETIME_DIFF(ce.charttime, ie.intime, HOUR) <= 24
  -- exclude rows marked as error
  and (ce.error IS NULL or ce.error = 0)
  RIGHT join sample_icu as si
  on ie.icustay_id = si.icustay_id
  where ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"

  -- Systolic/diastolic

  51, --	Arterial BP [Systolic]
  442, --	Manual BP [Systolic]
  455, --	NBP [Systolic]
  6701, --	Arterial BP #2 [Systolic]
  220179, --	Non Invasive Blood Pressure systolic
  220050, --	Arterial Blood Pressure systolic

  8368, --	Arterial BP [Diastolic]
  8440, --	Manual BP [Diastolic]
  8441, --	NBP [Diastolic]
  8555, --	Arterial BP #2 [Diastolic]
  220180, --	Non Invasive Blood Pressure diastolic
  220051, --	Arterial Blood Pressure diastolic


  -- MEAN ARTERIAL PRESSURE
  456, --"NBP Mean"
  52, --"Arterial BP Mean"
  6702, --	Arterial BP Mean #2
  443, --	Manual BP Mean(calc)
  220052, --"Arterial Blood Pressure mean"
  220181, --"Non Invasive Blood Pressure mean"
  225312, --"ART BP mean"

  -- RESPIRATORY RATE
  618,--	Respiratory Rate
  615,--	Resp Rate (Total)
  220210,--	Respiratory Rate
  224690, --	Respiratory Rate (Total)


  -- SPO2, peripheral
  646, 220277,

  -- GLUCOSE, both lab and fingerstick
  807,--	Fingerstick Glucose
  811,--	Glucose (70-105)
  1529,--	Glucose
  3745,--	BloodGlucose
  3744,--	Blood Glucose
  225664,--	Glucose finger stick
  220621,--	Glucose (serum)
  226537,--	Glucose (whole blood)

  -- TEMPERATURE
  223762, -- "Temperature Celsius"
  676,	-- "Temperature C"
  223761, -- "Temperature Fahrenheit"
  678 --	"Temperature F"
  )
  ORDER BY ie.icustay_id, 
  ce.charttime
"""

In [0]:
%%time
df_datapoint = client.query(query6_4, project=project_id).to_dataframe()
df_datapoint

CPU times: user 4min 43s, sys: 7.1 s, total: 4min 50s
Wall time: 9min 23s


In [0]:
#include data points from oxygenation and machine ventilation
query1 = """
WITH sample_icu AS(
SELECT subject_id,hadm_id,icustay_id,intime,outtime
FROM `mimiciii-database.MIMIC3_V1_4.ICUSTAYS` 
WHERE dbsource = 'metavision')
Select s.subject_id, s.hadm_id,v.icustay_id, s.intime, s.outtime, v.charttime, v.MechVent, v.OxygenTherapy
From `mimiciii-database.MIMIC3_V1_4.ventsettings` as v
RIGHT Join sample_icu as s 
ON v.icustay_id = s.icustay_id 
  and v.charttime between s.intime and s.outtime
  and DATETIME_DIFF(v.charttime, s.intime, SECOND) > 0
  and DATETIME_DIFF(v.charttime, s.intime, HOUR) <= 24
#WHERE v.MechVent = 0 AND (v.Extubated=1 OR v.SelfExtubated=1)
ORDER BY s.icustay_id, charttime
"""


In [0]:
df_02 = client.query(query1, project=project_id).to_dataframe()
#df_02

In [0]:
df_mv = df_02.iloc[:,:6]
df_mv['VitalID'] = 9
df_mv['valuenum'] = df_02['MechVent']
df_ot = df_02.iloc[:,:6]
df_ot['VitalID'] = 10
df_ot['valuenum'] = df_02['OxygenTherapy']
df_02transform = pd.concat([df_mv,df_ot])
df_02transform = df_02transform.sort_values(by=['icustay_id','charttime'])
df_02transform = df_02transform.dropna()
#df_02transform

In [0]:
# stratified icu admissions, db = metavision (2008-2012)
query4_2 = """
WITH adult_adm AS
( SELECT DISTINCT subject_id
  FROM `mimiciii-database.MIMIC3_V1_4.ADMISSIONS`
  WHERE admission_type != 'NEWBORN')
,
icu_mort AS
(
SELECT p.subject_id, i.hadm_id, i.icustay_id, i.intime, i.outtime, i.LOS, p.dod, dod_hosp, p.expire_flag,i.dbsource,
CASE 
  WHEN DATETIME(dod) BETWEEN intime AND outtime
    THEN 1
  WHEN DATETIME_DIFF(DATETIME(dod), outtime, HOUR) <= 24
    THEN 1
    ELSE 0
    END AS in_hospital_mortality
FROM `mimiciii-database.MIMIC3_V1_4.PATIENTS` p
JOIN `mimiciii-database.MIMIC3_V1_4.ICUSTAYS` i
ON p.subject_id = i.subject_id
    )
SELECT a.subject_id, im.hadm_id, im.icustay_id, im.intime, im.outtime, im.LOS, im.dod, im.dod_hosp, im.expire_flag,im.in_hospital_mortality
FROM adult_adm AS a
JOIN icu_mort AS im
ON a.subject_id = im.subject_id
WHERE im.LOS > 1  AND im.dbsource = 'metavision'
-- AND im.in_hospital_mortality = 1
"""
# ICU stay > 24hrs
#19705 ICU stays in total; 1623 in-hospital mortality. 8.23% mortality rate

In [0]:
df_icustay = client.query(query4_2, project=project_id).to_dataframe()
#df_icustay

In [0]:
#get icustay id
icu_stay = df_icustay['icustay_id']
icu_stay = icu_stay.values
#icu_stay = np.array(icu_stay, dtype=np.int64)
type(icu_stay),len(icu_stay)
#print(icu_stay)

In [0]:
# Calculate descriptive statistics of features
from collections import defaultdict
columns = ['HR','SBP','DBP','mean_BP','RR','BT','SpO2','Glu']
variables_s = defaultdict(dict)
for i,col in enumerate(columns):
    df_time1 = df_datapoint[df_datapoint['VitalID']==i+1]
    #df_time1 = df_time1.drop(columns=['VitalID'])
    #print(col)
    #print(df_time1['valuenum'].values)
    #print(len(df_time1['valuenum'].values))
    #print(df_time1['valuenum'].values.mean())
    #print(df_time1['valuenum'].values.std())
    variables_s[col] = {'mean':df_time1['valuenum'].values.mean(),'std':df_time1['valuenum'].values.std(), 'median':np.median(df_time1['valuenum'].values), 'max':df_time1['valuenum'].values.max(), 'min':df_time1['valuenum'].values.min()}
variables_s

defaultdict(dict,
            {'BT': {'max': 42.0,
              'mean': 36.7836548238141,
              'median': 36.77777777777778,
              'min': 15.0,
              'std': 0.8439861440166238},
             'DBP': {'max': 275.0,
              'mean': 61.77615497963075,
              'median': 60.0,
              'min': 4.0,
              'std': 14.943862856089247},
             'Glu': {'max': 948.0,
              'mean': 146.6260989981499,
              'median': 130.0,
              'min': 2.0,
              'std': 66.93611047066197},
             'HR': {'max': 280.0,
              'mean': 85.59267545134941,
              'median': 84.0,
              'min': 1.0,
              'std': 18.964264484217495},
             'RR': {'max': 69.0,
              'mean': 18.970714653234097,
              'median': 18.0,
              'min': 1.0,
              'std': 5.567947698995075},
             'SBP': {'max': 323.0,
              'mean': 118.37195048793568,
              'median': 116

In [0]:
# create data, label from samples, clean
# after sanity check

def create_data(dataset,icu_adm,parameters):
  data = []
  missing_data = []
  label = []
  columns = ['HR','SBP','DBP','mean_BP','RR','BT','SpO2','Glu']
  #check_tabel = pd.DataFrame(columns = ['icustay_id','HR','SBP','DBP','mean_BP','RR','BT','SpO2','Glu','mortality'])
  icu_stay = icu_adm['icustay_id']
  icu_stay = icu_stay.values
  for idx in tqdm_notebook(icu_stay):
    df_time = dataset[dataset['icustay_id']==idx]
    df_time = df_time.sort_values(by=['charttime'])
    df_time['charttime'] = pd.to_datetime(df_time['charttime'])
    df_time['relativetime'] = pd.to_datetime(df_time['charttime'])- df_time['intime'] + pd.to_datetime('2000/01/01')
            # extract datapoint by given time interval
    df_time_idx = df_time.set_index('relativetime')
            # group by hour
    q = df_time_idx.groupby([pd.Grouper(freq='60Min'),'VitalID'])
    q = q.first()['valuenum'].unstack()
    #missing_data.append(q.isnull().sum(axis=0).to_numpy())
    #l = df_icustay[df_icustay['icustay_id']==idx]['in_hospital_mortality']
    #data.append(q.values)
  #missing_data_per = np.array(missing_data)
  #print(type(missing_data_per))
  #missing_data_per = missing_data_per/24
  #average = missing_data_per.sum(axis=0)/len(sample)
    #fill the period gap with nan
    date = pd.to_datetime('2000-01-01 00:00:00')
    timesteps = date + pd.to_timedelta(np.arange(24), 'H')
    c = pd.DataFrame(columns = columns,index=timesteps)
          #matrix = []
    for step in timesteps:
      for i, v in enumerate(columns):
            #print(str(step))
        i+=1
        try:
                #matrix.append(q.loc[str(step),:].to_numpy())
          c.loc[str(step),v] = q.loc[str(step),i]

        except:
          #print(idx,str(step),v)
          pass
              #matrix.append(np.empty((8))*np.nan)
          #print(np.array(matrix))
          #print(np.isnan(np.array(matrix,dtype=np.float64)).sum(axis=0))
          #print(c.to_numpy())
          #print(c.isnull().sum(axis=0))
    #normalization
    #for var in columns:
    #        if var == 'SpO2':
    #          c[var] = (c[var]-variables_s[var]['mean'])/(variables_s[var]['max'] - variables_s[var]['min'])
    #        else:
    #          c[var] = (c[var]-variables_s[var]['mean'])/variables_s[var]['std']
          #fill missing data forward -> mean
    c = c.fillna(method='ffill').fillna(0)
    c = c.loc[:,parameters]
    #missing_data.append(c.isnull().sum(axis=0))
    data.append(c.to_numpy())
    #c['icustay_id'] = idx
    flag = icu_adm[icu_adm['icustay_id']==idx]['in_hospital_mortality']
    #print(flag)
    label.append(flag.values[0])
    #c['mortality'] = l.values[0]
    #check_tabel = pd.concat([check_tabel,c])
  #np.array(data)
  #check_tabel.to_csv('sample_checktable.csv')
  return np.array(data), np.array(label), icu_stay

In [0]:
#Create data
parameters=['HR','SBP','DBP','mean_BP','RR','BT','SpO2','Glu']
#df_icustay_100 = df_icustay.head(100)
#d, l, m = create_data(df_datapoint,df_icustay)
d, l, m = create_data(df_datapoint,df_icustay,parameters)

In [0]:
#Save data
my_data_dir = 'Colab Notebooks'
np.save("gdrive/My Drive/"+my_data_dir+"/outputs/first24hrs_nn.npy", d)
#np.save("gdrive/My Drive/"+my_data_dir+"/outputs/label_mort.npy", l)

In [0]:
# concate original data tabel with oxygenation and ventilation
df_datapoint_oxyg = pd.concat([df_datapoint,df_02transform])
#df_datapoint_oxyg

In [0]:
# create data, label from samples, clean
# after sanity check

def create_data_o2(dataset,icu_adm,parameters):

  data = []
  missing_data = []
  label = []
  columns = ['HR','SBP','DBP','mean_BP','RR','BT','SpO2','Glu','MV','O2']
  #check_tabel = pd.DataFrame(columns = ['icustay_id','HR','SBP','DBP','mean_BP','RR','BT','SpO2','Glu','mortality'])
  icu_stay = icu_adm['icustay_id']
  icu_stay = icu_stay.values
  for idx in tqdm_notebook(icu_stay):
    df_time = dataset[dataset['icustay_id']==idx]
    df_time = df_time.sort_values(by=['charttime'])
    df_time['charttime'] = pd.to_datetime(df_time['charttime'])
    df_time['relativetime'] = pd.to_datetime(df_time['charttime'])- df_time['intime'] + pd.to_datetime('2000/01/01')
            # extract datapoint by given time interval
    df_time_idx = df_time.set_index('relativetime')
            # group by hour
    q = df_time_idx.groupby([pd.Grouper(freq='60Min'),'VitalID'])
    q = q.first()['valuenum'].unstack()
    #missing_data.append(q.isnull().sum(axis=0).to_numpy())
    #l = df_icustay[df_icustay['icustay_id']==idx]['in_hospital_mortality']
    #data.append(q.values)
  #missing_data_per = np.array(missing_data)
  #print(type(missing_data_per))
  #missing_data_per = missing_data_per/24
  #average = missing_data_per.sum(axis=0)/len(sample)
    #fill the period gap with nan
    date = pd.to_datetime('2000-01-01 00:00:00')
    timesteps = date + pd.to_timedelta(np.arange(24), 'H')
    c = pd.DataFrame(columns = columns,index=timesteps)
          #matrix = []
    for step in timesteps:
      for i, v in enumerate(columns):
            #print(str(step))
        i+=1
        try:
                #matrix.append(q.loc[str(step),:].to_numpy())
          c.loc[str(step),v] = q.loc[str(step),i]

        except:
          #print(idx,str(step),v)
          pass
              #matrix.append(np.empty((8))*np.nan)
          #print(np.array(matrix))
          #print(np.isnan(np.array(matrix,dtype=np.float64)).sum(axis=0))
          #print(c.to_numpy())
          #print(c.isnull().sum(axis=0))
    #normalization
    for var in columns:
            if var == 'SpO2':
              c[var] = (c[var]-variables_s[var]['mean'])/(variables_s[var]['max'] - variables_s[var]['min'])
            elif var == 'MV': pass
            elif var == 'O2': pass
            else:
              #print(var)
              c[var] = (c[var]-variables_s[var]['mean'])/variables_s[var]['std']
          #fill missing data forward -> mean
    c = c.fillna(method='ffill').fillna(0)
    c = c.loc[:,parameters]
    #missing_data.append(c.isnull().sum(axis=0))
    data.append(c.to_numpy())
    #c['icustay_id'] = idx
    flag = icu_adm[icu_adm['icustay_id']==idx]['in_hospital_mortality']
    #print(flag)
    label.append(flag.values[0])
    #c['mortality'] = l.values[0]
    #check_tabel = pd.concat([check_tabel,c])
  #np.array(data)
  #check_tabel.to_csv('sample_checktable.csv')
  return np.array(data), np.array(label), icu_stay

In [0]:
#create dataset
parameters=['HR','SBP','DBP','mean_BP','RR','BT','SpO2','Glu','MV','O2']
#df_icustay_100 = df_icustay.head(100)
#d, l, m = create_data(df_datapoint,df_icustay)
d, l, m = create_data_o2(df_datapoint_oxyg,df_icustay,parameters)

In [0]:
# save dataset
my_data_dir = 'Colab Notebooks'
np.save("gdrive/My Drive/"+my_data_dir+"/outputs/first24hrs_o2.npy", d)
np.save("gdrive/My Drive/"+my_data_dir+"/outputs/label_mort_o2.npy", l)
np.save("gdrive/My Drive/"+my_data_dir+"/outputs/icustay_id_o2.npy", m)

In [0]:
my_data_dir = 'Colab Notebooks'
data = np.load("gdrive/My Drive/"+my_data_dir+"/outputs/first24hrs_o2.npy")
labels = np.load("gdrive/My Drive/"+my_data_dir+"/outputs/label_mort.npy")
id_list = np.load("gdrive/My Drive/"+my_data_dir+"/outputs/icustay_id.npy")